<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

Import Libraries, including BeautifulSoup which will be used to webscrape the Wikipedia page

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# we will use BeautifulSoup to webscrape the Toronto Postal Code Wikipedia page
!pip install beautifulsoup4
from urllib.request import urlopen
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


Webscrape the wikipedia page, then extract the postal code table

In [2]:
# Web scraping with BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

# Find the Postal Code table
tables = soup.find_all('table', class_='sortable')

# Extract the heading from the table
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]

# Extract the data in the table cells
dataarray=[] # Store the data in this array first
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    datarow = [td.text.strip() for td in tds]
    dataarray=np.append(dataarray,datarow)

#reshape the array into 2d array.  
num_col = len(headings) # The number of columns
num_row = len(dataarray)//num_col # number of row = length of dataarray divided by number of columns
dataarray=np.reshape(dataarray,(num_row,num_col))


Process the Data

In [3]:
# Create DataFrame
df=pd.DataFrame(dataarray,columns=headings)

# Remove rows where Borough is Not assigned
df = df[df.Borough != 'Not assigned']

# iterate through the rows, replace Not assigned Neighbourhood with Borough name
i=0
for j in df.iterrows():
    if(df.iloc[i].Neighbourhood=='Not assigned'):
        df.iloc[i].Neighbourhood=df.iloc[i].Borough
    i=i+1

# Combine the Neighbourhood grouped by Postcode and Borough
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

# Display the dataframe shape
print(df.shape)
print('\n\n\n\n')

# Display the dataframe
df

(103, 3)







,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
